# 4. Build the Neural Network

신경망은 데이터에 대한 연산을 수행하는 `계층(layer)`/`모듈(module)`로 구성되어 있습니다. <br><br>

* `torch.nn` 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다.
* PyTorch의 모든 모듈은 `nn.Module` 의 **하위 클래스(subclass)** 입니다. 
* 신경망은 다른 모듈(계층; layer)로 구성된 모듈입니다. 
>이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.



In [1]:
import os, torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 4.1 Get device for training

가능한 경우 GPU와 같은 하드웨어 가속기에서 모델을 학습하려고 합니다. `torch.cuda` 를 사용할 수 있는지 확인하고 그렇지 않으면 CPU를 계속 사용합니다.

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## 4.2 Define the class

신경망 모델을 `nn.Module` 의 하위클래스로 정의하고, `__init__` 에서 신경망 계층들을 초기화합니다. <br>
`nn.Module` 을 상속받은 모든 클래스는 `forward` 메소드에 **입력 데이터에 대한 연산**들을 구현합니다.

In [3]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

`NeuralNetwork` 의 인스턴스(instance)를 생성하고 이를 `device` 로 이동한 뒤, 구조(structure)를 출력합니다.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


* 모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 ***백그라운드 연산*** 들 과 함께 모델의 `forward` 를 실행합니다. <br>
model.forward() 를 직접 호출하지 마세요!

* 모델에 입력을 호출하면 각 분류(class)에 대한 raw 예측값이 있는 10-차원 텐서가 반환됩니다. <br>
원시 예측값을 `nn.Softmax` 모듈의 인스턴스에 통과시켜 **예측 확률**을 얻습니다.

In [7]:
# 임의의 input data 생성
X = torch.rand(1, 28, 28, device=device)
print(X.size())

torch.Size([1, 28, 28])


In [9]:
# 모델 예측값 확인 --> 정답 class가 10개 이기 때문에 size=10
logits = model(X)
print(logits.size())
print(logits)

torch.Size([1, 10])
tensor([[ 0.0409, -0.0656, -0.0115,  0.0100,  0.0104,  0.0456,  0.0821, -0.0063,
         -0.0431, -0.0502]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [10]:
# nn.Softmax 에 통과시켜 0~1 값으로 scaling
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)

tensor([[0.1039, 0.0934, 0.0986, 0.1008, 0.1008, 0.1044, 0.1083, 0.0992, 0.0956,
         0.0949]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [11]:
# 가장 높은 확률의 예측 class를 armax(1)로 얻는다
y_pred = pred_probab.argmax(1)

print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


## 4.3 Model Layers

FashionMNIST 모델의 계층들을 살펴보겠습니다. <br>
**28x28** 크기의 이미지 **3개**로 구성된 **미니배치**를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보겠습니다.

In [12]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

`nn.Flatten` 계층을 초기화하여 각 **28x28의 2D 이미지**를 **784 픽셀 값을 갖는 연속된 배열**로 변환합니다. (dim=0의 미니배치 차원은 유지됩니다.)

In [14]:
# Flatten 계층에 입력하면, 28*28 -> 784 로 차원 변환된 배열이 3개

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear<br>

**선형 계층**은 저장된 가중치(`weight`)와 편향(`bias`)을 사용하여 입력에 `선형 변환(linear transformation)`을 적용하는 모듈입니다.

In [15]:
# out_features 20 으로 linear transformation 됨을 확인

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [16]:
print(hidden1)

tensor([[-0.3676,  0.0197,  0.3481, -0.2340,  0.2281,  0.2402, -0.4320, -0.4275,
          0.2546,  0.0946,  0.0774, -0.2624, -0.0133, -0.0560, -0.0314, -0.0642,
         -0.4311, -0.0353,  0.1572, -0.4288],
        [-0.2757,  0.1584,  0.3598, -0.3118,  0.3493,  0.3552, -0.1772, -0.2064,
          0.2602, -0.1190,  0.2552, -0.2230, -0.0413, -0.0097,  0.2532,  0.0562,
         -0.2549,  0.0741, -0.0583, -0.6766],
        [-0.2370,  0.1495,  0.2945, -0.4485,  0.1517,  0.4344,  0.0203, -0.1688,
          0.0951, -0.5218,  0.1228,  0.0487,  0.1558, -0.1251,  0.4411, -0.3255,
         -0.2624,  0.1629, -0.0226, -0.5854]], grad_fn=<AddmmBackward0>)


### nn.ReLU

`비선형 활성화(activation)`는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만듭니다.<br> 비선형 활성화는 선형 변환 후에 적용되어 `비선형성(nonlinearity)` 을 도입하고, **신경망이 다양한 현상을 학습할 수 있도록 돕습니다**.

이 모델에서는 `nn.ReLU` 를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 **비선형성을 가진 다른 활성화를 도입할 수도 있습니다**.

In [17]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3676,  0.0197,  0.3481, -0.2340,  0.2281,  0.2402, -0.4320, -0.4275,
          0.2546,  0.0946,  0.0774, -0.2624, -0.0133, -0.0560, -0.0314, -0.0642,
         -0.4311, -0.0353,  0.1572, -0.4288],
        [-0.2757,  0.1584,  0.3598, -0.3118,  0.3493,  0.3552, -0.1772, -0.2064,
          0.2602, -0.1190,  0.2552, -0.2230, -0.0413, -0.0097,  0.2532,  0.0562,
         -0.2549,  0.0741, -0.0583, -0.6766],
        [-0.2370,  0.1495,  0.2945, -0.4485,  0.1517,  0.4344,  0.0203, -0.1688,
          0.0951, -0.5218,  0.1228,  0.0487,  0.1558, -0.1251,  0.4411, -0.3255,
         -0.2624,  0.1629, -0.0226, -0.5854]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0197, 0.3481, 0.0000, 0.2281, 0.2402, 0.0000, 0.0000, 0.2546,
         0.0946, 0.0774, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1572, 0.0000],
        [0.0000, 0.1584, 0.3598, 0.0000, 0.3493, 0.3552, 0.0000, 0.0000, 0.2602,
         0.0000, 0.2552, 0.0000, 0.0000, 0.0000, 0.25

## nn.Sequential

`nn.Sequential` 은 순서를 갖는 모듈의 컨테이너입니다. <br>
데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. sequential container를 사용하여 아래의 `seq_modules` 와 같은 신경망을 빠르게 만들 수 있습니다.

In [18]:
seq_modules = nn.Sequential(
    flatten,  # nn.Flatten()
    layer1,  # nn.Linear(in_features=28*28, out_features=20) 
    nn.ReLU(),
    nn.Linear(20, 10) # out_features 20, 10 classifications 이므로
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [19]:
# 3개의 이미지에 대한 예측값
print(logits)

tensor([[-0.0717,  0.1054, -0.1870, -0.1946,  0.2715, -0.1940, -0.0677,  0.0563,
          0.3319, -0.1045],
        [-0.0029,  0.1813, -0.2657, -0.2504,  0.2861, -0.1500, -0.0234,  0.0385,
          0.2824, -0.0393],
        [-0.0353,  0.1967, -0.2932, -0.2645,  0.3049, -0.2063,  0.1366,  0.0869,
          0.2446, -0.1796]], grad_fn=<AddmmBackward0>)


## nn.Softmax

신경망의 마지막 선형 계층은 `nn.Softmax` 모듈에 전달될 ([-infty, infty] 범위의 raw value인 logits 를 반환합니다.<br> logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)됩니다. dim 매개변수는 값의 합이 1이 되는 차원을 나타냅니다.

In [20]:
# nn.Softmax로 raw logits 0-1 scaling

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0920, 0.1098, 0.0820, 0.0814, 0.1297, 0.0814, 0.0924, 0.1046, 0.1378,
         0.0890],
        [0.0974, 0.1171, 0.0749, 0.0760, 0.1300, 0.0841, 0.0954, 0.1015, 0.1296,
         0.0939],
        [0.0945, 0.1192, 0.0730, 0.0751, 0.1328, 0.0796, 0.1122, 0.1068, 0.1250,
         0.0818]], grad_fn=<SoftmaxBackward0>)


## 4.4 Model Parameters

신경망 내부의 많은 계층들은 `매개변수화(parameterize)` 됩니다. <br>
즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다. 
<br><br>
`nn.Module` 을 상속하면 모델 객체 내부의 **모든 필드들이 자동으로 추적(track)**되며, 모델의 `parameters()` 및 `named_parameters()` 메소드로 모든 매개변수에 접근할 수 있게 됩니다.

이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력합니다.

In [21]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0086,  0.0123,  0.0026,  ..., -0.0227, -0.0247, -0.0143],
        [-0.0039, -0.0032, -0.0164,  ..., -0.0295, -0.0044,  0.0028]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0235, -0.0325], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0333,  0.0259, -0.0176,  ...,  0.0172, -0.0349,  0.0050],
        [ 0.0026,  0.0421, -0.0366,  ...,  0.0280,  0.0075,  0.0389]],
       device='cuda:0', grad_fn=<Slice